##  Graphviz networks  plotted with Plotly ## 
 ###  &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; U.S Open binary tree via Networkx, Pygraphviz and Plotly ###

During the last year we illustrated how Plotly can visualize networks  defined by different Python libraries and of various layouts:

- layout assigned by `igraph` or `networkx` : [https://plot.ly/~empet/9110/](https://plot.ly/~empet/9110/coauthorship-network-of-scientists-working-on-network-theory-and-experiment-data/)
- custom layout: [https://plot.ly/~empet/13574/](https://plot.ly/~empet/13574/arc-diagram-of-star-wars-characters-that-interacted-in-the-force-awakens/)
- prescribed layout,  giving node positions in the network data file: [https://plot.ly/~empet/12908/human-disease-network/](https://plot.ly/~empet/12908/human-disease-network/)
- particular Gephi [layout](https://pypi.python.org/pypi/ForceAtlas2/1.0) for the `numpy` network: [https://plot.ly/~empet/13902/](https://plot.ly/~empet/13902/numpy-cluster-in-the-network-of-python-packages/)
- unrooted tree defined via `biopython` with an `igraph` layout: [https://plot.ly/~empet/13876/](https://plot.ly/~empet/13876/unrooted-tree-from-sequence-alignments-with-biopython-igraph-and-plotly/) 

`igraph` and `networkx` provide  layouts mostly based on different layout algorithms, and the networks generated by these  Python libraries are plotted via `cairo`, respectively
`matplotlib`. Neither `igraph` nor `networkx` can handle and plot large networks. 

An appropriate layout for large networks is provided by [`graphviz`](http://www.graphviz.org/).  Moreover `graphviz` and  `pygraphviz`, a Python interface to `graphviz`, have graph layouts that are not implemented in `igraph`, and `networkx`.
[Here](https://plot.ly/~empet/14003/price-network-1/) is illustrated the so called Price network, created through a preferential attachement, and plotted with `graphviz` layout `sfdp`, destinated to large networks.

Using Python Plotly we can process  network data with one or more  of the above mentioned libraries,  assign any layout in the collection of all their layouts, and get an interactive plot.

In this Jupyter notebook we illustrate the complementary use of `networkx`, `pygraphviz` (with `graphviz` in the backend), and Plotly, to generate the radial tree representing the evolution of women players at U.S. Open 2016.

 US Open is a tennis tournament attended by $N=128=2^7$ women players. It is a  balanced knokout tournament, consisting in 7 rounds.
 A few days before the start the organizers release a draw, that points  out  the pairs that play in the first round.
Then succesivelly, the  $(2k+1)^{th}$ and $(2k+2)^{th}$ winners in a round $r\geq 1$  play together in the $(r+1)^{th}$ round,
 $k=0, ..., 2^{7-r}$.
After the $6^{th}$ round remain only two players and the winner of their final match is the winner of the Grand Slam. The whole process is visualized by [WTA](http://www.wtatennis.com/SEWTATour-Archive/posting/2016/905/MDS.pdf) as a balanced binary tree, having as root the tournament's winner, and the players in all rounds  are the tree nodes. The parent of a pair  $({2k+1}, 2k+2)$ playing in a round r is the winner of the corresponding match.

To define and plot the corresponding binary tree we proceed as follows:
 - setup an Excel file from the binary tree posted at WTA [site](http://www.wtatennis.com/SEWTATour-Archive/posting/2016/905/MDS.pdf).
The players are indexed traversing the tree by the [Breadth First Search](https://en.wikipedia.org/wiki/Breadth-first_search) method.
 - the `networkx.balanced_tree` function creates the binary tree G.
 - the nodes and edges of the graph G define  a `pygraphviz` graph H, with the radial layout, called `twopi`.
 - the node positions assigned by pygraphviz layout, as well as the corresponding edges are used to define the attributes of the Plotly objects that represent
    the binary tree.

In [1]:
import networkx as nx
import pygraphviz as pgv
import pandas as pd
import numpy as np

Read the Excel file:

In [47]:
df = pd.read_excel("Data/US-Open-2016.xls")
df.ix[:6, :]#print tree levels 0, 1, 2 

,name
0,Kerber
1,Pliskova
2,Kerber
3,S Williams
4,Pliskova
5,Wozniacki
6,Kerber


In [48]:
N=len(df)
N

127

In [49]:
labels=list(df['name'])

Define  the tree $G$ as a `networkx` graph:

In [50]:
G=nx.balanced_tree(2, 6)
V=G.nodes()
E=G.edges()

`pygraphviz` tree H, and its layout are defined below:

In [51]:
H=pgv.AGraph(strict=True, directed=False)
H.add_nodes_from(V)
H.add_edges_from(E)
H.layout(prog='twopi') 

The function `position` extracts the node coordinates of the `pygraphviz` tree $H$:

In [52]:
def position(g):
    if  not isinstance(g, pgv.AGraph):
        raise ValueError('The graph g must be a pygraphviz AGraph')
    N=len(g.nodes())    
    pos=[]
    for k in range(N):
        s=g.get_node(k).attr['pos']
        t=s.split(",")
        pos.append(map(float, t))
    return pos    

Process the above defined network data to get the corresponding Plotly plot of the binary tree:

In [53]:
import plotly.plotly as py
from plotly.graph_objs import *

The Plotly version of a graph of edges E and  node coordinates `pos` is returned by the following function:

In [54]:
def plotly_graph(E, pos):
    # E is the list of tuples representing the graph edges
    # pos is the list of node coordinates 
    N=len(pos)
    Xn=[pos[k][0] for k in range(N)]# x-coordinates of nodes
    Yn=[pos[k][1] for k in range(N)]# y-coordnates of nodes

    Xe=[]
    Ye=[]
    for e in E:
        Xe+=[pos[e[0]][0],pos[e[1]][0], None]# x coordinates of the nodes defining the edge e
        Ye+=[pos[e[0]][1],pos[e[1]][1], None]# y - " - 
        
    return Xn, Yn, Xe, Ye    

Get  node positions in the tree H:

In [55]:
pos=position(H)
#Rotate position with pi/2 anti-clockwise
pos=np.array(pos)
pos[:,[0, 1]] = pos[:,[1, 0]]
pos[:,0]=-pos[:,0]

Define the Plotly objects that represent the  binary tree, and the finalist routes  to the last match: 

In [56]:
Xn, Yn, Xe, Ye=plotly_graph(E, pos)

edges=Scatter(x=Xe,
               y=Ye, 
               mode='lines',
               line=Line(color='rgb(160,160,160)', width=0.75),
               hoverinfo='none'
               )
nodes=Scatter(x=Xn, 
               y=Yn,
               mode='markers',
               name='',
               marker=Marker(symbol='dot',
                             size=8, 
                             color='#85b6b6', 
                             line=Line(color='rgb(100,100,100)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

In [57]:
Kerber_path=[0, 2, 6, 14, 30, 62, 126]
Pliskova_path=[1, 4, 10, 21, 43, 87]
colorKP=['#CC0000']*len(Kerber_path)+['rgb(65, 64, 123)']*len(Pliskova_path)# set color for both paths

In [58]:
XK=[pos[k][0] for k in Kerber_path]
YK=[pos[k][1] for k in Kerber_path]
XP=[pos[k][0] for k in Pliskova_path]
YP=[pos[k][1] for k in Pliskova_path]

finalists=Scatter(x=XK+XP, 
                  y=YK+YP, 
                  mode='markers', 
                  marker=dict(symbol='dot',
                              size=10, 
                              color=colorKP, 
                              line=Line(color='rgb(100,100,100)', width=0.5),                             
                             ),
                  text=['Kerber']*len(Kerber_path)+ ['Pliskova']*len(Pliskova_path), 
                  hoverinfo='text'  
                 )

We attach to each player  in the second round its name aligned radially with respect to the corresponding node position. The function `set_annotation` places the names at
their  position, with text displayed at a given angle:

In [59]:
def set_annotation(x, y, anno_text,  textangle, fontsize=11, color='rgb(10,10,10)'): 
  
    return Annotation(x= x,  
                      y= y,       
                      text= anno_text,      
                      textangle=textangle,#angle with horizontal line through (x,y), in degrees;
                                          #+ =clockwise, -=anti-clockwise
                      font= Font(size=fontsize, color=color),  
                      showarrow=False     
                     ) 

Define Plotly plot layout:

In [60]:
axis=dict(showbackground=False,
          showline=False,  
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='' 
          )

In [61]:
layout = Layout(
         title="U.S. Open 2016<br>Radial binary tree associated to women's singles  players", 
         font=dict(family='Balto'),
         width=650,
         height=650,
         showlegend=False,
         xaxis=XAxis(axis),
         yaxis=YAxis(axis), 
         margin=Margin(t=100),
         hovermode='closest',
                )

The node positions returned by  `pygraphviz` radial  layout are not located on a circle centered at origin.
That is why we calculate the circle center and radius:

In [62]:
center=np.array([(np.min(pos[63:, 0])+np.max(pos[63:, 0]))/2, (np.min(pos[63:, 1])+np.max(pos[63:, 1]))/2])
radius=np.linalg.norm(pos[63,:]-center)

Compute the text angle:

In [63]:
angles=[]
for k in range(63, 127):
    v=pos[k,:]-center
    angles.append(-(180*np.arctan(v[1]/v[0])/np.pi))

In [64]:
pos_text=center+1.2*(pos[63:, :]-center)# text position

#define annotations for non-finalist players
for k in range(63, 87):
    layout['annotations']+=[set_annotation(pos_text[k-63][0], pos_text[k-63][1], labels[k],  angles[k-63])]
for k in range(88, 126):
    layout['annotations']+=[set_annotation(pos_text[k-63][0], pos_text[k-63][1], labels[k],  angles[k-63])]    

#insert colored annotations for the finalists, Pliskova and Kerber
layout['annotations']+=[set_annotation(pos_text[87-63][0], pos_text[87-63][1], 
                                       '<b>Pliskova</b>', 
                                       angles[87-63], 
                                       color='rgb(65, 64, 123)'),
                         set_annotation(pos_text[126-63][0], pos_text[126-63][1], 
                                        '<b>Kerber</b>',  
                                        angles[126-63], 
                                        color='#CC0000')]
layout['annotations']+= [set_annotation(center[0]-0.15, center[1]+45, 
                                        '<b>Winner<br>A. Kerber</b>', 
                                        0, fontsize=12, color='#CC0000')]

Append the annotation that displays data source:

In [65]:
data_anno_text="Data source: "+\
               "<a href='http://www.wtatennis.com/SEWTATour-Archive/posting/2016/905/MDS.pdf'> [1] </a>,"+\
               " Excel file: "+\
                "<a href='https://github.com/empet/Plotly-plots/blob/master/Data/US-Open-2016.xls'> [2] </a>"
layout['annotations'].append(Annotation(
            showarrow=False, 
            text=data_anno_text,  
            xref='paper',     
            yref='paper',     
            x=0,  
            y=-0.1,  
            xanchor='left',   
            yanchor='bottom',  
            font=Font(
            size=12 )
            ))

In [66]:
data=Data([edges, nodes, finalists])
fig=Figure(data=data, layout=layout)

In [67]:
py.sign_in('empet', 'my_api_key')
py.iplot(fig, filename='US-Open-16')

In [68]:
from IPython.core.display import HTML
def  css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()